In [1]:
import numpy as np
import pandas as pd
from mmcv import load, dump

data1 = load('/home/pilab/ActionRecognition/gmskl/01_skeleton_extraction/my_skeleton/B/1/fold_1_test.pkl')
data2 = load('/home/pilab/ActionRecognition/gmskl/01_skeleton_extraction/my_skeleton/B/2/fold_1_test.pkl')
data3 = load('/home/pilab/ActionRecognition/gmskl/01_skeleton_extraction/my_skeleton/B/3/fold_1_test.pkl')
data4 = load('/home/pilab/ActionRecognition/gmskl/01_skeleton_extraction/my_skeleton/B/4/fold_1_test.pkl')

/home/pilab/ActionRecognition/gmskl/.venv/lib64/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data1

[{'keypoint': array([[[[ 241.79788  ,  359.5028   ],
           [ 250.46036  ,  333.5152   ],
           [ 241.79788  ,  324.85266  ],
           ...,
           [ 189.82266  ,  879.2545   ],
           [ 146.51004  ,  957.2173   ],
           [ 155.17258  , 1052.5051   ]],
  
          [[ 251.59232  ,  361.7459   ],
           [ 251.59232  ,  335.81958  ],
           [ 242.95016  ,  335.81958  ],
           ...,
           [ 182.45535  ,  880.2731   ],
           [ 208.38168  ,  888.91516  ],
           [ 156.52902  , 1053.1154   ]],
  
          [[ 253.89594  ,  359.92743  ],
           [ 262.46564  ,  334.21838  ],
           [ 253.89594  ,  334.21838  ],
           ...,
           [ 193.90808  ,  874.1092   ],
           [ 219.61719  ,  916.9577   ],
           [ 151.05963  , 1045.5032   ]],
  
          ...,
  
          [[ 978.8274   ,   -1.5921326],
           [ 994.749    ,  -17.513733 ],
           [ 973.52014  ,  -12.206543 ],
           ...,
           [ 915.141    ,  369.91

In [3]:
def organize(dt, actnum):
    df = pd.DataFrame(columns=['pid', f'act{actnum}'])
    for i in range(len(dt)):
        pid = dt[i]['frame_dir'][:4]
        df.loc[i] = [pid, dt[i]['frame_dir']]

    return df

In [4]:
test_1 = organize(data1, 1)
test_2 = organize(data2, 2)
test_3 = organize(data3, 3)
test_4 = organize(data4, 4)

In [5]:
test_1

,pid,act1
0,B124,B124_GMS_1_1_2
1,B063,B063_GMS_1_1_1
2,B163,B163_GMS_1_1_4
3,B077,B077_GMS_1_1_5
4,B183,B183_GMS_1_1_3
...,...,...
111,B137,B137_GMS_1_1_5
112,B157,B157_GMS_1_1_4
113,B063,B063_GMS_1_1_5
114,B070,B070_GMS_1_1_1


In [6]:
import random

pidLi = np.array(test_1['pid'].sort_values().unique())
newdf = pd.DataFrame(columns=['pid', 'act1', 'act2', 'act3', 'act4'])
prev_arr = newdf
for item in pidLi:
    arr1 = test_1[test_1['pid'] == item].reset_index().drop(columns=['index'])
    arr2 = test_2[test_2['pid'] == item].drop(columns=['pid']).reset_index().drop(columns=['index'])
    arr3 = test_3[test_3['pid'] == item].drop(columns=['pid']).reset_index().drop(columns=['index'])
    arr4 = test_4[test_4['pid'] == item].drop(columns=['pid']).reset_index().drop(columns=['index'])

    cat_arr = pd.concat([arr1, arr2, arr3, arr4], axis=1)
    cat_arr.dropna(how='any', inplace=True)

    arr1_sz = len(arr1.dropna())
    arr2_sz = len(arr2.dropna())
    arr3_sz = len(arr3.dropna())
    arr4_sz = len(arr4.dropna())
    min_len = min(arr1_sz, arr2_sz, arr3_sz, arr4_sz)
    max_len = max(arr1_sz, arr2_sz, arr3_sz, arr4_sz)

    last_two_rows_index = cat_arr.index[-2:]
    if ((max_len - min_len) > 3) :
        cat_arr.drop(last_two_rows_index)

    cat_arr = cat_arr.apply(lambda x: x.ffill() if x.name == 'pid' else (x.fillna(random.sample(list(x.dropna()), 1)[0]) if len(list(x.dropna()))>0 else x))
    # print(cat_arr)
    print(prev_arr)
    print(cat_arr)
    new_arr = pd.concat([prev_arr, cat_arr], axis=0)
    prev_arr = new_arr

Empty DataFrame
Columns: [pid, act1, act2, act3, act4]
Index: []
    pid            act1            act2             act3            act4
0  B011  B011_GMS_1_1_1  B011_GMS_2_1_2   B011_GMS_3_1_4  B011_GMS_4_1_8
1  B011  B011_GMS_1_1_4  B011_GMS_2_1_3   B011_GMS_3_1_5  B011_GMS_4_1_2
2  B011  B011_GMS_1_1_3  B011_GMS_2_1_5   B011_GMS_3_1_8  B011_GMS_4_1_4
3  B011  B011_GMS_1_1_5  B011_GMS_2_1_9  B011_GMS_3_1_10  B011_GMS_4_1_1
4  B011  B011_GMS_1_1_2  B011_GMS_2_1_6   B011_GMS_3_1_7  B011_GMS_4_1_3
5  B011  B011_GMS_1_1_6  B011_GMS_2_1_8   B011_GMS_3_1_2  B011_GMS_4_1_9
6  B011  B011_GMS_1_1_8  B011_GMS_2_1_1   B011_GMS_3_1_3  B011_GMS_4_1_7
7  B011  B011_GMS_1_1_9  B011_GMS_2_1_4   B011_GMS_3_1_6  B011_GMS_4_1_5
8  B011  B011_GMS_1_1_7  B011_GMS_2_1_7   B011_GMS_3_1_9  B011_GMS_4_1_6
    pid            act1            act2             act3            act4
0  B011  B011_GMS_1_1_1  B011_GMS_2_1_2   B011_GMS_3_1_4  B011_GMS_4_1_8
1  B011  B011_GMS_1_1_4  B011_GMS_2_1_3   B011_GMS_3_1_5  B

In [7]:
overall_label = pd.read_csv('../02_labels/overall_label.csv')

In [14]:
each_label = pd.read_csv('../02_labels/gms_eval_v3.csv')

In [8]:
overall_label

,ID,label_pid
0,B001,0
1,B006,1
2,B007,0
3,B011,0
4,B014,1
...,...,...
136,B201,0
137,B202,0
138,B203,0
139,B204,0


In [16]:
each_label = each_label[['ID', 'action', '1차 평가']]

In [18]:
each_label = each_label[each_label['ID'].str[0] == 'B']

In [20]:
each_label['action'] = each_label['action'].str[:5] + '_1' + each_label['action'].str[5:]

/home/pilab/ActionRecognition/gmskl/.venv/lib64/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [23]:
each_label

,ID,action,1차 평가
0,B001,B001_GMS_1_1_1,3.0
1,B001,B001_GMS_1_1_2,3.0
2,B001,B001_GMS_1_1_3,3.0
3,B001,B001_GMS_1_1_4,3.0
4,B001,B001_GMS_1_1_5,3.0
...,...,...,...
2497,B205,B205_GMS_4_1_1,3.0
2498,B205,B205_GMS_4_1_2,3.0
2499,B205,B205_GMS_4_1_3,3.0
2500,B205,B205_GMS_4_1_4,3.0


In [22]:
each_label['action'] = each_label['ID'] + '_' +  each_label['action']

/home/pilab/ActionRecognition/gmskl/.venv/lib64/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [36]:
each_label = each_label[['action', '1차 평가']]

In [33]:
final = pd.merge(prev_arr, overall_label, left_on='pid', right_on='ID')

In [34]:
final = final.drop(columns=['ID'])

In [35]:
final

,pid,act1,act2,act3,act4,label_pid
0,B011,B011_GMS_1_1_1,B011_GMS_2_1_2,B011_GMS_3_1_4,B011_GMS_4_1_8,0
1,B011,B011_GMS_1_1_4,B011_GMS_2_1_3,B011_GMS_3_1_5,B011_GMS_4_1_2,0
2,B011,B011_GMS_1_1_3,B011_GMS_2_1_5,B011_GMS_3_1_8,B011_GMS_4_1_4,0
3,B011,B011_GMS_1_1_5,B011_GMS_2_1_9,B011_GMS_3_1_10,B011_GMS_4_1_1,0
4,B011,B011_GMS_1_1_2,B011_GMS_2_1_6,B011_GMS_3_1_7,B011_GMS_4_1_3,0
...,...,...,...,...,...,...
98,B183,B183_GMS_1_1_1,B183_GMS_2_1_2,B183_GMS_3_1_2,B183_GMS_4_1_2,0
99,B183,B183_GMS_1_1_4,B183_GMS_2_1_1,B183_GMS_3_1_3,B183_GMS_4_1_4,0
100,B183,B183_GMS_1_1_2,B183_GMS_2_1_4,B183_GMS_3_1_5,B183_GMS_4_1_3,0
101,B186,B186_GMS_1_1_1,B186_GMS_2_1_1,B186_GMS_3_1_2,B186_GMS_4_1_1,1


In [43]:
def merge_by_action(action):
    target = action
    act_tmp = final[[target]]
    act_tmp = pd.merge(act_tmp, each_label, left_on=target, right_on='action', how='left')
    act_tmp.drop(columns=['action'], inplace=True)
    return act_tmp

In [44]:
act1 = merge_by_action('act1')
act2 = merge_by_action('act2')
act3 = merge_by_action('act3')
act4 = merge_by_action('act4')

In [55]:
final2 = pd.concat([final['pid'],act1, act2, act3, act4, final['label_pid']], axis=1)

In [56]:
final2.to_csv('test_data_info.csv', index=False)